# Environment sanity check

Run this cell first.

It verifies that:
- Required Python packages are installed
- Plotly `FigureWidget` works
- ipywidgets render correctly
- Core FK helper functions execute

If anything fails, restart the kernel and re-run. On Binder, failures usually mean the environment build did not complete yet.


In [ ]:
import importlib, sys

required = ["numpy", "plotly", "ipywidgets", "sympy"]

print("Checking required packages...")
missing = []

for pkg in required:
    if importlib.util.find_spec(pkg) is None:
        missing.append(pkg)
    else:
        print(f"✓ {pkg}")

if missing:
    print("\nMissing packages:", missing)
    print("If running locally, install them via pip.")
else:
    print("\nAll required packages found.")

# --- Widget test ---
import ipywidgets as widgets
from IPython.display import display

print("\nTesting ipywidgets rendering...")
display(widgets.IntSlider(description="Widget test"))

# --- Plotly FigureWidget test ---
import plotly.graph_objects as go

print("Testing Plotly FigureWidget...")
fig = go.FigureWidget(
    data=[go.Scatter3d(x=[0,1], y=[0,1], z=[0,1], mode="lines+markers")]
)
fig.update_layout(title="Plotly sanity check")
display(fig)

# --- FK math sanity check ---
import numpy as np

def Rz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def T(R,p):
    Tm = np.eye(4)
    Tm[:3,:3] = R
    Tm[:3,3] = p
    return Tm

theta = np.deg2rad(30)
T_test = T(Rz(theta), np.array([1,0,0]))

print("\nFK transform test ^0T_1 =")
print(np.round(T_test, 4))

print("\nSanity check complete.")


# Forward Kinematics in 3D — Interactive Binder Workbook

This workbook continues from the planar FK notebook and moves to **3D forward kinematics** using **4×4 homogeneous transforms**.

You will interact with 3D robots and **see coordinate frames** move as you change joint angles.

## Learning goals
- Build and multiply **3D homogeneous transforms** for serial chains
- Understand the difference between **world axes** and **local (body) axes**
- Read end-effector pose as **position + orientation**
- Verify derivations numerically and (optionally) symbolically

---


## Binder notes

This notebook uses:
- `numpy`
- `plotly` (3D interactive visualization in the browser)
- `ipywidgets` (sliders and toggles)
- `sympy` (optional symbolic checks)

If widgets do not render in your environment, see the installation cell below (safe on Binder, but typically unnecessary).


In [ ]:
# If widgets don't appear, uncomment these lines (Binder / fresh environments):
# %pip -q install numpy plotly ipywidgets sympy
# %pip -q install jupyterlab jupyterlab_widgets

import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display


# 1) Core math: rotations and homogeneous transforms

In 3D, we represent rigid transforms with a 4×4 homogeneous matrix:

$$
{}^{A}T_{B} =
\begin{bmatrix}
{}^{A}R_{B} & {}^{A}p_{B} \\
0\ 0\ 0 & 1
\end{bmatrix},
$$

where ${}^{A}R_{B}$ is a 3×3 rotation matrix and ${}^{A}p_{B}$ is a 3×1 translation vector (the origin of frame $B$ expressed in $A$).

To compose transforms along a chain:

$$
{}^{0}T_{n} = {}^{0}T_{1}\, {}^{1}T_{2}\, \cdots\, {}^{n-1}T_{n}.
$$

Below are helper functions for $R_x(\theta)$, $R_y(\theta)$, $R_z(\theta)$ and for building 4×4 transforms.


In [ ]:
def Rx(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0, 0],
                     [0, c, -s],
                     [0, s,  c]])

def Ry(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])

def Rz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])

def T(R, p):
    """Create a 4x4 homogeneous transform from rotation R (3x3) and translation p (3,)."""
    Tm = np.eye(4)
    Tm[:3, :3] = R
    Tm[:3, 3] = p
    return Tm

def trans(x, y, z):
    return T(np.eye(3), np.array([x, y, z], dtype=float))

def rot_x(theta):
    return T(Rx(theta), np.zeros(3))

def rot_y(theta):
    return T(Ry(theta), np.zeros(3))

def rot_z(theta):
    return T(Rz(theta), np.zeros(3))

def apply_T(Tm, p):
    """Apply 4x4 transform to 3D point p (3,)."""
    ph = np.array([p[0], p[1], p[2], 1.0])
    qh = Tm @ ph
    return qh[:3]

def pose_from_T(Tm):
    """Return (R, p) from transform."""
    return Tm[:3, :3], Tm[:3, 3]


## Visualizing frames in 3D

We will draw:
- **Links** as line segments between joint origins
- **Frames** as 3 short axis lines (x, y, z)

For readability, we keep axes uncolored by default. The notebook labels each axis as `x`, `y`, `z` near its tip.

(If you want colored axes, it’s a one-line change per trace.)


In [ ]:
def frame_traces(origin, R, axis_len=0.25, name_prefix=""):
    """Create Plotly traces for a coordinate frame at origin with rotation R."""
    o = origin
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    x_tip = o + axis_len * ex
    y_tip = o + axis_len * ey
    z_tip = o + axis_len * ez

    traces = []
    traces.append(go.Scatter3d(x=[o[0], x_tip[0]], y=[o[1], x_tip[1]], z=[o[2], x_tip[2]],
                               mode="lines", name=f"{name_prefix}x"))
    traces.append(go.Scatter3d(x=[o[0], y_tip[0]], y=[o[1], y_tip[1]], z=[o[2], y_tip[2]],
                               mode="lines", name=f"{name_prefix}y"))
    traces.append(go.Scatter3d(x=[o[0], z_tip[0]], y=[o[1], z_tip[1]], z=[o[2], z_tip[2]],
                               mode="lines", name=f"{name_prefix}z"))
    traces.append(go.Scatter3d(x=[x_tip[0]], y=[x_tip[1]], z=[x_tip[2]], mode="text", text=["x"], showlegend=False))
    traces.append(go.Scatter3d(x=[y_tip[0]], y=[y_tip[1]], z=[y_tip[2]], mode="text", text=["y"], showlegend=False))
    traces.append(go.Scatter3d(x=[z_tip[0]], y=[z_tip[1]], z=[z_tip[2]], mode="text", text=["z"], showlegend=False))
    return traces

def link_trace(points, name="links"):
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    zs = [p[2] for p in points]
    return go.Scatter3d(x=xs, y=ys, z=zs, mode="lines+markers", name=name)

def make_3d_figure(xlim=2.5, ylim=2.5, zlim=2.5, title=""):
    fig = go.FigureWidget()
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(range=[-xlim, xlim], title="x"),
            yaxis=dict(range=[-ylim, ylim], title="y"),
            zaxis=dict(range=[-zlim, zlim], title="z"),
            aspectmode="cube"
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=False,
    )
    return fig


# 2) 1-DOF in 3D: a single revolute joint about $z$

We start with a simple 3D case that *looks* planar: a revolute joint about the world $z$ axis, and a link of length $L_1$ along the joint's local $x$ axis.

The transform is:

$$
{}^{0}T_{1} = \text{Rot}_z(\theta_1)\, \text{Trans}(L_1, 0, 0).
$$

Use the slider to explore how rotating about $z$ moves the link in the $xy$ plane.


In [ ]:
def fk_1Rz(theta1, L1):
    T01 = rot_z(theta1) @ trans(L1, 0, 0)
    R01, p01 = pose_from_T(T01)
    return T01, [np.zeros(3), p01]

fig1 = make_3d_figure(xlim=2.5, ylim=2.5, zlim=2.5, title="1-DOF (Revolute about z)")
fig1.add_trace(link_trace([np.zeros(3), np.array([1.0,0,0])], name="link"))

for tr in frame_traces(np.zeros(3), np.eye(3), axis_len=0.35, name_prefix="0_"):
    fig1.add_trace(tr)
for tr in frame_traces(np.array([1.0,0,0]), np.eye(3), axis_len=0.35, name_prefix="1_"):
    fig1.add_trace(tr)

theta1_slider = widgets.FloatSlider(value=30.0, min=-180, max=180, step=1, description="θ1 (deg)")
L1_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L1")
show_frames = widgets.Checkbox(value=True, description="Show frames")
out_pose = widgets.Output()

def update_1dof(theta1_deg, L1, show_frames=True):
    theta1 = np.deg2rad(theta1_deg)
    T01, pts = fk_1Rz(theta1, L1)
    p_end = pts[-1]

    with fig1.batch_update():
        fig1.data[0].x = [pts[0][0], pts[1][0]]
        fig1.data[0].y = [pts[0][1], pts[1][1]]
        fig1.data[0].z = [pts[0][2], pts[1][2]]

        end_start = 1 + 6  # [link] + base frame traces (6)
        R01, p01 = pose_from_T(T01)
        end_trs = frame_traces(p01, R01, axis_len=0.35, name_prefix="1_")

        for i in range(6):
            fig1.data[end_start + i].x = end_trs[i].x
            fig1.data[end_start + i].y = end_trs[i].y
            fig1.data[end_start + i].z = end_trs[i].z
            fig1.data[end_start + i].mode = end_trs[i].mode
            fig1.data[end_start + i].text = getattr(end_trs[i], "text", None)

        for i in range(1, 1+12):
            fig1.data[i].visible = show_frames

    with out_pose:
        out_pose.clear_output()
        print("End-effector position p = ", np.round(p_end, 4))
        print("^0T_1 =")
        print(np.round(T01, 4))

ui = widgets.VBox([widgets.HBox([theta1_slider, L1_slider]), show_frames])
widgets.interactive_output(update_1dof, {"theta1_deg": theta1_slider, "L1": L1_slider, "show_frames": show_frames})
display(ui, fig1, out_pose)
update_1dof(theta1_slider.value, L1_slider.value, show_frames.value)


# 3) 2-DOF in 3D: non-parallel joint axes (order matters)

We will use:
- Joint 1: revolute about **world** $z$ (a yaw)
- Joint 2: revolute about **local** $y$ of frame {1} (a pitch)

$$
{}^{0}T_{1} = \text{Rot}_z(\theta_1)\, \text{Trans}(L_1, 0, 0),\qquad
{}^{1}T_{2} = \text{Rot}_y(\theta_2)\, \text{Trans}(L_2, 0, 0)
$$
$$
{}^{0}T_{2} = {}^{0}T_{1}\,{}^{1}T_{2}.
$$


In [ ]:
def fk_2R_zy(theta1, theta2, L1, L2):
    T01 = rot_z(theta1) @ trans(L1, 0, 0)
    T12 = rot_y(theta2) @ trans(L2, 0, 0)
    T02 = T01 @ T12

    p0 = np.zeros(3)
    p1 = pose_from_T(T01)[1]
    p2 = pose_from_T(T02)[1]
    return (T01, T12, T02), [p0, p1, p2]

fig2 = make_3d_figure(xlim=3.0, ylim=3.0, zlim=3.0, title="2-DOF (z then local y)")
fig2.add_trace(link_trace([np.zeros(3), np.array([1.0,0,0]), np.array([2.0,0,0])]))

for tr in frame_traces(np.zeros(3), np.eye(3), axis_len=0.35, name_prefix="0_"):
    fig2.add_trace(tr)
for tr in frame_traces(np.array([1.0,0,0]), np.eye(3), axis_len=0.35, name_prefix="1_"):
    fig2.add_trace(tr)
for tr in frame_traces(np.array([2.0,0,0]), np.eye(3), axis_len=0.35, name_prefix="2_"):
    fig2.add_trace(tr)

theta1 = widgets.FloatSlider(value=20.0, min=-180, max=180, step=1, description="θ1 (deg)")
theta2 = widgets.FloatSlider(value=30.0, min=-180, max=180, step=1, description="θ2 (deg)")
L1 = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L1")
L2 = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L2")
show_frames2 = widgets.Checkbox(value=True, description="Show frames")
out_pose2 = widgets.Output()

def update_2dof(theta1_deg, theta2_deg, L1_val, L2_val, show_frames=True):
    th1 = np.deg2rad(theta1_deg)
    th2 = np.deg2rad(theta2_deg)
    (T01, T12, T02), pts = fk_2R_zy(th1, th2, L1_val, L2_val)
    R01, p1 = pose_from_T(T01)
    R02, p2 = pose_from_T(T02)

    with fig2.batch_update():
        fig2.data[0].x = [pts[0][0], pts[1][0], pts[2][0]]
        fig2.data[0].y = [pts[0][1], pts[1][1], pts[2][1]]
        fig2.data[0].z = [pts[0][2], pts[1][2], pts[2][2]]

        f1 = 1 + 6
        f2 = 1 + 12

        f1_trs = frame_traces(p1, R01, axis_len=0.35, name_prefix="1_")
        f2_trs = frame_traces(p2, R02, axis_len=0.35, name_prefix="2_")

        for i in range(6):
            fig2.data[f1 + i].x = f1_trs[i].x
            fig2.data[f1 + i].y = f1_trs[i].y
            fig2.data[f1 + i].z = f1_trs[i].z
            fig2.data[f1 + i].mode = f1_trs[i].mode
            fig2.data[f1 + i].text = getattr(f1_trs[i], "text", None)

            fig2.data[f2 + i].x = f2_trs[i].x
            fig2.data[f2 + i].y = f2_trs[i].y
            fig2.data[f2 + i].z = f2_trs[i].z
            fig2.data[f2 + i].mode = f2_trs[i].mode
            fig2.data[f2 + i].text = getattr(f2_trs[i], "text", None)

        for i in range(1, 1+18):
            fig2.data[i].visible = show_frames

    with out_pose2:
        out_pose2.clear_output()
        print("End-effector position p = ", np.round(p2, 4))
        print("^0T_2 =")
        print(np.round(T02, 4))

ui2 = widgets.VBox([widgets.HBox([theta1, theta2]), widgets.HBox([L1, L2]), show_frames2])
widgets.interactive_output(update_2dof, {
    "theta1_deg": theta1, "theta2_deg": theta2, "L1_val": L1, "L2_val": L2, "show_frames": show_frames2
})
display(ui2, fig2, out_pose2)
update_2dof(theta1.value, theta2.value, L1.value, L2.value, show_frames2.value)


## Orientation readouts (rotation matrix, roll–pitch–yaw, quaternion)

We use the ZYX (yaw–pitch–roll) convention for RPY extraction:

$$
R = R_z(\psi)\, R_y(\theta)\, R_x(\phi),
$$

where $(\phi,\theta,\psi)$ are (roll, pitch, yaw). RPY has singularities; quaternions avoid them.


In [ ]:
def rpy_from_R_zyx(R):
    yaw = np.arctan2(R[1,0], R[0,0])
    pitch = np.arctan2(-R[2,0], np.sqrt(R[2,1]**2 + R[2,2]**2))
    roll = np.arctan2(R[2,1], R[2,2])
    return roll, pitch, yaw

def quat_from_R(R):
    tr = np.trace(R)
    if tr > 0:
        S = np.sqrt(tr + 1.0) * 2
        w = 0.25 * S
        x = (R[2,1] - R[1,2]) / S
        y = (R[0,2] - R[2,0]) / S
        z = (R[1,0] - R[0,1]) / S
    else:
        if (R[0,0] > R[1,1]) and (R[0,0] > R[2,2]):
            S = np.sqrt(1.0 + R[0,0] - R[1,1] - R[2,2]) * 2
            w = (R[2,1] - R[1,2]) / S
            x = 0.25 * S
            y = (R[0,1] + R[1,0]) / S
            z = (R[0,2] + R[2,0]) / S
        elif R[1,1] > R[2,2]:
            S = np.sqrt(1.0 + R[1,1] - R[0,0] - R[2,2]) * 2
            w = (R[0,2] - R[2,0]) / S
            x = (R[0,1] + R[1,0]) / S
            y = 0.25 * S
            z = (R[1,2] + R[2,1]) / S
        else:
            S = np.sqrt(1.0 + R[2,2] - R[0,0] - R[1,1]) * 2
            w = (R[1,0] - R[0,1]) / S
            x = (R[0,2] + R[2,0]) / S
            y = (R[1,2] + R[2,1]) / S
            z = 0.25 * S
    q = np.array([w, x, y, z])
    q = q / np.linalg.norm(q)
    return q

# Demo using current 2-DOF pose
(T01, T12, T02), pts = fk_2R_zy(np.deg2rad(theta1.value), np.deg2rad(theta2.value), L1.value, L2.value)
R02, p2 = pose_from_T(T02)
roll, pitch, yaw = rpy_from_R_zyx(R02)
q = quat_from_R(R02)

print("R (rounded):\n", np.round(R02, 4))
print("RPY ZYX (deg): roll={:.2f}, pitch={:.2f}, yaw={:.2f}".format(np.rad2deg(roll), np.rad2deg(pitch), np.rad2deg(yaw)))
print("quat (w,x,y,z):", np.round(q, 4))


# 4) 3R “elbow” arm in 3D (yaw–pitch–pitch)

A common educational 3D arm is a 3R elbow-like structure:
- Joint 1: yaw about $z$
- Joint 2: pitch about local $y$
- Joint 3: pitch about local $y$ again

$$
{}^{0}T_{1} = \text{Rot}_z(\theta_1)\, \text{Trans}(L_1, 0, 0)
$$
$$
{}^{1}T_{2} = \text{Rot}_y(\theta_2)\, \text{Trans}(L_2, 0, 0)
$$
$$
{}^{2}T_{3} = \text{Rot}_y(\theta_3)\, \text{Trans}(L_3, 0, 0)
$$
$$
{}^{0}T_{3} = {}^{0}T_{1}\, {}^{1}T_{2}\, {}^{2}T_{3}.
$$


In [ ]:
def fk_3R_zyy(theta1, theta2, theta3, L1, L2, L3):
    T01 = rot_z(theta1) @ trans(L1, 0, 0)
    T12 = rot_y(theta2) @ trans(L2, 0, 0)
    T23 = rot_y(theta3) @ trans(L3, 0, 0)
    T02 = T01 @ T12
    T03 = T02 @ T23

    p0 = np.zeros(3)
    p1 = pose_from_T(T01)[1]
    p2 = pose_from_T(T02)[1]
    p3 = pose_from_T(T03)[1]
    return (T01, T12, T23, T03), [p0, p1, p2, p3]

fig3 = make_3d_figure(xlim=3.5, ylim=3.5, zlim=3.5, title="3R elbow arm (z-y-y)")
fig3.add_trace(link_trace([np.zeros(3), np.array([1.0,0,0]), np.array([2.0,0,0]), np.array([3.0,0,0])]))

for tr in frame_traces(np.zeros(3), np.eye(3), axis_len=0.30, name_prefix="0_"):
    fig3.add_trace(tr)
for tr in frame_traces(np.array([1.0,0,0]), np.eye(3), axis_len=0.30, name_prefix="1_"):
    fig3.add_trace(tr)
for tr in frame_traces(np.array([2.0,0,0]), np.eye(3), axis_len=0.30, name_prefix="2_"):
    fig3.add_trace(tr)
for tr in frame_traces(np.array([3.0,0,0]), np.eye(3), axis_len=0.30, name_prefix="3_"):
    fig3.add_trace(tr)

th1 = widgets.FloatSlider(value=20.0, min=-180, max=180, step=1, description="θ1 (deg)")
th2 = widgets.FloatSlider(value=30.0, min=-180, max=180, step=1, description="θ2 (deg)")
th3 = widgets.FloatSlider(value=-20.0, min=-180, max=180, step=1, description="θ3 (deg)")
L1w = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L1")
L2w = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L2")
L3w = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="L3")
show_frames3 = widgets.Checkbox(value=True, description="Show frames")
out_pose3 = widgets.Output()

def update_3R(th1_deg, th2_deg, th3_deg, L1_val, L2_val, L3_val, show_frames=True):
    a1, a2, a3 = np.deg2rad(th1_deg), np.deg2rad(th2_deg), np.deg2rad(th3_deg)
    (T01, T12, T23, T03), pts = fk_3R_zyy(a1, a2, a3, L1_val, L2_val, L3_val)
    R01, p1 = pose_from_T(T01)
    R02, p2 = pose_from_T(T01 @ T12)
    R03, p3 = pose_from_T(T03)

    with fig3.batch_update():
        fig3.data[0].x = [p[0] for p in pts]
        fig3.data[0].y = [p[1] for p in pts]
        fig3.data[0].z = [p[2] for p in pts]

        base = 1
        f1 = base + 6
        f2 = base + 12
        f3 = base + 18

        f1_trs = frame_traces(p1, R01, axis_len=0.30, name_prefix="1_")
        f2_trs = frame_traces(p2, R02, axis_len=0.30, name_prefix="2_")
        f3_trs = frame_traces(p3, R03, axis_len=0.30, name_prefix="3_")

        for i in range(6):
            fig3.data[f1 + i].x = f1_trs[i].x
            fig3.data[f1 + i].y = f1_trs[i].y
            fig3.data[f1 + i].z = f1_trs[i].z
            fig3.data[f1 + i].mode = f1_trs[i].mode
            fig3.data[f1 + i].text = getattr(f1_trs[i], "text", None)

            fig3.data[f2 + i].x = f2_trs[i].x
            fig3.data[f2 + i].y = f2_trs[i].y
            fig3.data[f2 + i].z = f2_trs[i].z
            fig3.data[f2 + i].mode = f2_trs[i].mode
            fig3.data[f2 + i].text = getattr(f2_trs[i], "text", None)

            fig3.data[f3 + i].x = f3_trs[i].x
            fig3.data[f3 + i].y = f3_trs[i].y
            fig3.data[f3 + i].z = f3_trs[i].z
            fig3.data[f3 + i].mode = f3_trs[i].mode
            fig3.data[f3 + i].text = getattr(f3_trs[i], "text", None)

        for i in range(1, 1+24):
            fig3.data[i].visible = show_frames

    with out_pose3:
        out_pose3.clear_output()
        roll, pitch, yaw = rpy_from_R_zyx(R03)
        q = quat_from_R(R03)
        print("End-effector position p = ", np.round(p3, 4))
        print("Rotation matrix R (rounded):\n", np.round(R03, 4))
        print("RPY ZYX (deg): roll={:.2f}, pitch={:.2f}, yaw={:.2f}".format(np.rad2deg(roll), np.rad2deg(pitch), np.rad2deg(yaw)))
        print("quat (w,x,y,z):", np.round(q, 4))
        print("^0T_3 =\n", np.round(T03, 4))

ui3 = widgets.VBox([widgets.HBox([th1, th2, th3]), widgets.HBox([L1w, L2w, L3w]), show_frames3])
widgets.interactive_output(update_3R, {
    "th1_deg": th1, "th2_deg": th2, "th3_deg": th3,
    "L1_val": L1w, "L2_val": L2w, "L3_val": L3w,
    "show_frames": show_frames3
})
display(ui3, fig3, out_pose3)
update_3R(th1.value, th2.value, th3.value, L1w.value, L2w.value, L3w.value, show_frames3.value)


# 5) Optional: Denavit–Hartenberg (DH) parameters (classic convention)

Classic DH writes each link transform as:

$$
{}^{i-1}T_i =
\text{Rot}_z(\theta_i)\,
\text{Trans}_z(d_i)\,
\text{Trans}_x(a_i)\,
\text{Rot}_x(\alpha_i).
$$


In [ ]:
def dh_classic(theta, d, a, alpha):
    return rot_z(theta) @ trans(0, 0, d) @ trans(a, 0, 0) @ rot_x(alpha)

def fk_from_dh(dh_rows):
    Ts = [np.eye(4)]
    Tcur = np.eye(4)
    for (theta, d, a, alpha) in dh_rows:
        Tcur = Tcur @ dh_classic(theta, d, a, alpha)
        Ts.append(Tcur)
    return Ts

theta1_demo = np.deg2rad(30)
theta2_demo = np.deg2rad(-20)
dh_demo = [
    (theta1_demo, 0.2, 1.0, np.deg2rad(90)),
    (theta2_demo, 0.0, 1.0, np.deg2rad(0)),
]
Ts = fk_from_dh(dh_demo)
print("^0T_2 (rounded):\n", np.round(Ts[-1], 4))


# 6) Optional symbolic check (SymPy)

This section verifies the 2-DOF chain symbolically:

$$
{}^{0}T_{2} = (\text{Rot}_z(t_1)\,\text{Trans}(L_1,0,0))\,(\text{Rot}_y(t_2)\,\text{Trans}(L_2,0,0)).
$$


In [ ]:
import sympy as sp

t1, t2, L1s, L2s = sp.symbols("t1 t2 L1 L2", real=True)

def rotz_sym(theta):
    c, s = sp.cos(theta), sp.sin(theta)
    return sp.Matrix([[c, -s, 0],
                      [s,  c, 0],
                      [0,  0, 1]])

def roty_sym(theta):
    c, s = sp.cos(theta), sp.sin(theta)
    return sp.Matrix([[ c, 0, s],
                      [ 0, 1, 0],
                      [-s, 0, c]])

def trans_sym(x,y,z):
    return sp.Matrix([[1,0,0,x],
                      [0,1,0,y],
                      [0,0,1,z],
                      [0,0,0,1]])

def T_sym(R, p):
    return sp.Matrix([[R[0,0], R[0,1], R[0,2], p[0]],
                      [R[1,0], R[1,1], R[1,2], p[1]],
                      [R[2,0], R[2,1], R[2,2], p[2]],
                      [0,0,0,1]])

T01 = T_sym(rotz_sym(t1), sp.Matrix([0,0,0])) * trans_sym(L1s, 0, 0)
T12 = T_sym(roty_sym(t2), sp.Matrix([0,0,0])) * trans_sym(L2s, 0, 0)
T02 = sp.simplify(T01 * T12)

p02 = sp.Matrix([T02[0,3], T02[1,3], T02[2,3]])
p02


# Exercises (recommended)

1. **Local vs world axis**: Modify the 2-DOF model so joint 2 rotates about **world** $y$ instead of local $y$.
2. **Order matters**: Swap rotation/translation order in one joint and describe the effect.
3. **3R reachability**: Fix $\theta_1=0$ and vary $\theta_2,\theta_3$ — what 3D surface does the end-effector trace?
4. **DH challenge**: Write a DH table for a simple arm and verify with `dh_classic`.

---
